In [3]:
pip install pillow requests


Note: you may need to restart the kernel to use updated packages.


In [7]:
import tkinter as tk
from tkinter import messagebox
from tkinter import PhotoImage
import requests
from PIL import Image, ImageTk
import io
from datetime import datetime

# ---------------- Configuration ----------------
API_KEY = "55a3cbb795dd4719c6f799cc4e6bb99f"  # Replace with your OpenWeatherMap API key

# ---------------- App Class ----------------
class DoodleWeatherApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Doodle Weather")
        self.root.configure(bg="#e0f7ff")
        self.root.geometry("400x650")
        self.root.resizable(False, False)

        self.setup_ui()

    def setup_ui(self):
        # Main card frame
        self.card = tk.Frame(self.root, bg="white", bd=0, relief=tk.FLAT)
        self.card.place(relx=0.5, rely=0.5, anchor=tk.CENTER, width=350, height=580)

        # Title
        self.title = tk.Label(self.card, text="☀️ Doodle Weather", font=("Helvetica", 16, "bold"), bg="white", fg="#222222")
        self.title.pack(pady=(20, 10))

        # Entry and search
        entry_frame = tk.Frame(self.card, bg="white")
        entry_frame.pack(pady=(0, 20))
        self.entry = tk.Entry(entry_frame, font=("Helvetica", 12), bd=0, relief=tk.FLAT, width=20)
        self.entry.pack(side=tk.LEFT, ipady=6, padx=(0, 5))
        self.entry.insert(0, "Enter location")

        search_btn = tk.Button(entry_frame, text="🔍", command=self.search_weather, font=("Helvetica", 12), bd=0, bg="white")
        search_btn.pack(side=tk.LEFT)

        # Weather icon and temp frame
        self.icon_canvas = tk.Canvas(self.card, width=300, height=150, bg="white", bd=0, highlightthickness=0)
        self.icon_canvas.pack()

        self.temp_label = tk.Label(self.card, text="--°", font=("Helvetica", 36, "bold"), bg="white", fg="#333333")
        self.temp_label.pack(pady=(10, 0))

        self.desc_label = tk.Label(self.card, text="Condition", font=("Helvetica", 16), bg="white", fg="#555555")
        self.desc_label.pack()

        # Info cards frame
        self.info_frame = tk.Frame(self.card, bg="white")
        self.info_frame.pack(pady=20)

        self.humidity_card = self.create_info_card(self.info_frame, "#52d18d", "Humidity", "--%")
        self.humidity_card.grid(row=0, column=0, padx=5)

        self.wind_card = self.create_info_card(self.info_frame, "#3bb4f2", "Wind", "-- km/h")
        self.wind_card.grid(row=0, column=1, padx=5)

        self.pressure_card = self.create_info_card(self.info_frame, "#686de0", "Pressure", "-- hPa")
        self.pressure_card.grid(row=0, column=2, padx=5)

        self.visibility_card = self.create_info_card(self.info_frame, "#f55d7a", "Visibility", "-- km")
        self.visibility_card.grid(row=0, column=3, padx=5)

        # Refresh button
        self.refresh_btn = tk.Button(self.card, text="🔄 Refresh", command=self.refresh_weather, font=("Helvetica", 14, "bold"),
                                     bg="#ff6e7f", fg="white", activebackground="#ff6e7f", bd=0, relief=tk.FLAT, cursor="hand2")
        self.refresh_btn.pack(pady=10, ipadx=20, ipady=6)

    def create_info_card(self, parent, color, label, value):
        frame = tk.Frame(parent, bg=color, width=70, height=70, bd=0)
        frame.pack_propagate(False)
        value_label = tk.Label(frame, text=value, font=("Helvetica", 10, "bold"), fg="white", bg=color)
        value_label.pack()
        label_label = tk.Label(frame, text=label, font=("Helvetica", 8), fg="white", bg=color)
        label_label.pack()
        setattr(self, f"{label.lower()}_value", value_label)
        return frame

    def search_weather(self):
        location = self.entry.get()
        if not location or location == "Enter location":
            messagebox.showwarning("Warning", "Please enter a location.")
            return
        self.get_weather(location)

    def refresh_weather(self):
        location = self.entry.get()
        if location and location != "Enter location":
            self.get_weather(location)

    def get_weather(self, location):
        try:
            geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={location}&limit=1&appid={API_KEY}"
            geo_response = requests.get(geo_url).json()
            if not geo_response:
                messagebox.showerror("Error", "Location not found.")
                return

            lat = geo_response[0]["lat"]
            lon = geo_response[0]["lon"]

            weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
            weather_response = requests.get(weather_url).json()

            # Update UI
            temp = weather_response["main"]["temp"]
            desc = weather_response["weather"][0]["description"].title()
            icon_code = weather_response["weather"][0]["icon"]

            humidity = weather_response["main"]["humidity"]
            wind = weather_response["wind"]["speed"] * 3.6
            pressure = weather_response["main"]["pressure"]
            visibility = weather_response.get("visibility", 0) / 1000

            self.temp_label.config(text=f"{int(temp)}°")
            self.desc_label.config(text=desc)

            self.humidity_value.config(text=f"{humidity}%")
            self.wind_value.config(text=f"{wind:.1f} km/h")
            self.pressure_value.config(text=f"{pressure} hPa")
            self.visibility_value.config(text=f"{visibility:.1f} km")

            # Load icon
            self.load_icon(icon_code)

        except Exception as e:
            print(e)
            messagebox.showerror("Error", "Could not fetch weather data.")

    def load_icon(self, code):
        try:
            icon_url = f"https://openweathermap.org/img/wn/{code}@2x.png"
            response = requests.get(icon_url, stream=True)
            img_data = response.content
            img = Image.open(io.BytesIO(img_data)).resize((100, 100))
            photo = ImageTk.PhotoImage(img)
            self.icon_canvas.delete("all")
            self.icon_canvas.create_image(150, 75, image=photo)
            self.icon_canvas.image = photo
        except:
            self.icon_canvas.delete("all")

# ---------------- Main ----------------
if __name__ == "__main__":
    root = tk.Tk()
    app = DoodleWeatherApp(root)
    root.mainloop()
